In [1]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

gray_cmap = plt.colormaps.get('Greys')
gray_cmap.set_bad('lightblue')

In [2]:
SMALL_SIZE = 6
MEDIUM_SIZE = 8
BIGGER_SIZE = 10

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Decomposition results

## SVD
Plot the explained variance of the SVD decompositions

In [3]:
#Dictionary with the paths to SVD results and the number of most important components.
svd = {
    Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\PhaseMappingPaper\Data\Dataset A\datasetA_preprocessed_SVD1.npz'): 5,
    Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\PhaseMappingPaper\Data\Dataset A\datasetA_preprocessed_SVD2.npz'): 6,
    Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\PhaseMappingPaper\Data\Dataset B\datasetB_preprocessed_SVD1.npz'): 5,
    Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\PhaseMappingPaper\Data\Dataset B\datasetB_preprocessed_SVD2.npz'): 6
}

In [4]:
n = 21 #Number of components to show
components = np.arange(n)+1 #The component labelling.

#Plot/figure parameters
dpi=300 #Figure resolution
textwidth = 468 #Page width in pt
axes_per_line = 3 #Number of axes/figures to put on each line in final figure - used for sizing each individual plot
figwidth = textwidth/axes_per_line #The width of each figure
figheight = figwidth #The height of each figure
pt2in = 0.01389 #Conversion from points to inches
figsize = (figwidth*pt2in, figheight*pt2in) #Figure size in inches
signal_fmt={'marker': 'o', 'color': 'k', 'markerfacecolor': 'k', 'lw': 0, 'ms':4} #Format for the most important components in the scree-plot
noise_fmt={'marker':'o', 'color': 'k', 'markerfacecolor': 'w', 'lw': 0, 'ms':4} #Format for the other components in the scree-plot

#Plot the explained variance ratio of the SVD decompositions
for svd_path in svd:
    s = np.load(svd_path, allow_pickle=True) #Load a SVD decomposition
    threshold = svd[svd_path] #Get the threshold for the SVD decomposition (used for highlightning the most important components in the scree-plot    

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi) #Get a figure with a single axis
    ax.semilogy(components[:threshold], s['explained_variance_ratio'][:threshold], **signal_fmt) #plot the most important components
    ax.semilogy(components[threshold:n], s['explained_variance_ratio'][threshold:n], **noise_fmt) #plot the other components
    ax.set_xlabel(f'Principal component number') #Set x-axis label
    ax.set_xticks(components[::5]) #Set x-axis ticks
    ax.set_ylabel(f'Proportion of variance') #Set y-axis label
    plt.tight_layout() #Force tight layout
    
    fig.savefig(svd_path.with_name(f'{svd_path.stem}_scree.png')) #Save figure

## NMF
Plot the NMF components

In [5]:
#Paths to NMF decomposition results
nmf = [
    Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\PhaseMappingPaper\Data\Dataset A\datasetA_preprocessed_NMF1.npz'),
    Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\PhaseMappingPaper\Data\Dataset A\datasetA_preprocessed_NMF2.npz'),
    Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\PhaseMappingPaper\Data\Dataset B\datasetB_preprocessed_NMF1.npz'),
    Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\PhaseMappingPaper\Data\Dataset B\datasetB_preprocessed_NMF2.npz')
]

In [6]:
#Plot/figure parameters
dpi=300 #Figure resolution
textwidth = 468 #Page width in pt
axes_per_line = 3 #Number of axes/figures to put on each line in final figure - used for sizing each individual plot
figwidth = textwidth/axes_per_line #The width of each figure
figheight = figwidth #The height of each figure
pt2in = 0.01389 #Conversion from points to inches
figsize = (figwidth*pt2in, figheight*pt2in) #Figure size in inches
inset_size = 0.25 #Inset size given as fraction of figure size

#Plot the loadings of each NMF decomposition with the factor inset in the upper right corner. Non-square loadings/factors will give poor/unexpected results
for path in nmf:
    s = np.load(path, allow_pickle=True) #Load an NMF decomposition series
    factors = s['factors'].reshape(*s['original_shape'][2:], s['output_dimension']) #Extract the factors and reshape them (they are stored as (ndx*ndy, N))
    loadings = s['loadings'].reshape(*s['original_shape'][:2], s['output_dimension']) #Extract the loadings and reshape them (they are stored as (nx*ny, N))
    #Make individual figures and plots for each loading and inset the factor in the upper right corner
    for i in range(loadings.shape[-1]):
        fig = plt.figure(figsize=figsize, dpi=dpi) #Create a new figure
        ax = fig.add_axes([0, 0, 1, 1], xticks = [], yticks = [], frameon=False) #Add a clean and empty axis
        ax.imshow(loadings[:, :, i], cmap=gray_cmap) #Show the loading
        inset_ax = fig.add_axes([1-inset_size, 1-inset_size, inset_size, inset_size], xticks=[], yticks=[], frameon=False) #Add a clean and empty axis in the upper right corner
        inset_ax.imshow(factors[:, :, i], cmap=gray_cmap) #Show the factor
        fig.savefig(path.with_name(f'{path.stem}_{i}.png')) #Save the figure
        plt.close(fig) #Close the figure
    